In [327]:
from neo import Block
from neo.io import Spike2IO, NeoMatlabIO
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle
from kaveh.behavioral import oculomotor
from kaveh.sorting import spikesorter
from kaveh.toolbox import find_file
from kaveh.plots import axvlines
import numpy as np
from matplotlib import pyplot as plt
from smr import File
import csv
import os
import gc

In [328]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [408]:
# f_name = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_25deg/B082107/B082107_1340_List.smr'
f_name = '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Buckley_25deg/B110807/B110807_1632_List.smr'

In [409]:
neo_reader = Spike2IO(filename=f_name)
neo_data = neo_reader.read()
data_block = neo_data[0]

seg = data_block.segments[0]

events = seg.events

In [412]:
# Load events
labels = np.unique(events[-1].labels)

event_times = [0]*labels.size

for i, label in enumerate(labels):
    event_times[i] = events[-1].times[events[-1].labels == label]
    
    
# Event 0
# Event 1
# Event 2
# Event 3 : Trial onset
# Event 4 : Primary target presentation in adaptation trials
# Event 5 : Corrective target presentation in adaptation trials
# Event 6 : Divide each trial into its target jumps
# Event 7 : 

In [411]:
for et in event_times:
    print(et.shape)

(5605,)
(651,)


In [407]:
events[7]

Event
name: 'Events'
annotations: {'comment': u'No comment',
  'copy': False,
  'id': u'31',
  'physical_channel_index': 0}

In [394]:
# Load eye data
HE = seg.analogsignals[0].as_array()
t_HE = seg.analogsignals[0].times;

VE = seg.analogsignals[1].as_array()
t_VE = seg.analogsignals[1].times

HT = seg.analogsignals[2].as_array()
t_HT = seg.analogsignals[2].times

VT = seg.analogsignals[3].as_array()
t_VT = seg.analogsignals[3].times

eye_fs = seg.analogsignals[0].sampling_rate

In [386]:
type(t_VT)

quantities.quantity.Quantity

In [395]:
# Load voltage
signal = seg.analogsignals[-1].as_array()
t_signal = seg.analogsignals[-1].times
signal_fs = seg.analogsignals[-1].sampling_rate

In [413]:
class trial:
    def __init__(self, HT, t_HT, VT, t_VT, HE, t_HE, VE, t_VE, signal, t_signal, trial_number,
                 start_time, end_time):
        '''
        HT, VT, HE, VE, signal: Type ndarray
        HT_t, VT_t, HE_t, VE_t, t_signal: Type Quantity ('s')
        trial_number: Integer
        start_time, end_time: In seconds (type: Quantity ('s'))
        '''
        self.HT = HT
        self.t_HT = t_HT
        self.VT = VT
        self.t_VT = t_VT
        self.HE = HE
        self.t_HE = t_HE
        self.VE = VE
        self.t_VE = t_VE
        self.signal
        self.t_signal = t_signal
        self.trial_number = trial_number
        self.start_time = start_time
        self.end_time = end_time


In [414]:
prim_sac_times_adpt = event_times[4]
corr_sac_times_adpt = event_times[5]

IndexError: list index out of range

In [391]:
# divide trials
trial_num = 0

start_time = event_times[3][trial_num]
end_time = event_times[3][trial_num + 1]

gc.collect()
plt.figure()
plt.plot(t_VT[(t_VT > start_time) & (t_VT <= end_time)], VT[(t_VT > start_time) & (t_VT <= end_time)], alpha = 0.5, color = 'm')
plt.plot(t_HT[(t_HT > start_time) & (t_HT <= end_time)], HT[(t_HT > start_time) & (t_HT <= end_time)], alpha = 0.5, color = 'b')
plt.plot(t_VE[(t_VE > start_time) & (t_VE <= end_time)], VE[(t_VE > start_time) & (t_VE <= end_time)], alpha = 0.5, color = 'g')
plt.plot(t_HE[(t_HE > start_time) & (t_HE <= end_time)], HE[(t_HE > start_time) & (t_HE <= end_time)], alpha = 0.5, color = 'y')
plt.legend(['VT', 'HT', 'VE', 'HE'])
plt.ylim((-20, 20))
plt.eventplot(prim_sac_times_adpt[(prim_sac_times_adpt > start_time) & (prim_sac_times_adpt <= end_time)], linelengths=40, colors='r', alpha = 0.3)
plt.eventplot(corr_sac_times_adpt[(corr_sac_times_adpt > start_time) & (corr_sac_times_adpt <= end_time)], linelengths=40, colors='lime', alpha = 0.3)
plt.show()

IndexError: index 1 is out of bounds for axis 0 with size 1